In [1]:
from SymbolicDSGE import ModelConfig, ModelParser, DSGESolver
import sympy as sp
from warnings import catch_warnings, simplefilter
import numpy as np

In [11]:
conf = ModelParser("MODELS/test.yaml").get()

In [12]:
with catch_warnings():
    simplefilter(action="ignore")
    mat = sp.Matrix(conf.equations.model)
mat

Matrix([
[                Eq(Pi(t), beta*Pi(t + 1) + kappa*x(t))],
[Eq(x(t), -sigma*(-Pi(t + 1) + r(t)) + u(t) + x(t + 1))],
[                              Eq(u(t + 1), rho_u*u(t))],
[Eq(r_star(t), phi_pi*Pi(t) + phi_x*x(t) + rbar + v(t))],
[  Eq(r(t + 1), rho_r*r(t) + (1 - rho_r)*r_star(t + 1))],
[                              Eq(v(t + 1), rho_v*v(t))]])

In [13]:
sol = DSGESolver(conf)
comp = sol.compile(variable_order=conf.variables, n_state=3, n_exog=2)
conf.variables

[u, v, r, Pi, x, r_star]

In [14]:
solved = sol.solve(
    comp,
    steady_state=np.asarray([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=float),
    log_linear=False,
)

C:\Users\guney\Documents\GitHub\ThesisDraft\SymbolicDSGE\solver.py:229: ComplexWarning: Casting complex values to real discards the imaginary part
  p = np.asarray(mdl.p, dtype=float64)
C:\Users\guney\Documents\GitHub\ThesisDraft\SymbolicDSGE\solver.py:230: ComplexWarning: Casting complex values to real discards the imaginary part
  f = np.asarray(mdl.f, dtype=float64)


In [15]:
solved.policy.eig

array([5.29640671e-01+0.j, 6.99999918e-01+0.j, 4.99999963e-01+0.j,
       5.03611337e+16+0.j, 1.45329116e+00+0.j, 1.03671150e+00+0.j])

In [36]:
e_u = np.random.normal(0, 0.5, size=(1,))
e_v = np.random.normal(0, 0.25, size=(1,))
shocks = np.array([[e_u[0], e_v[0]]])

params = {
    p.name: float(conf.calibration.parameters[p])
    for p in conf.parameters
    if p in conf.calibration.parameters
}

cur = solved.sim(1, shocks)["_X"][0]
fwd = solved.sim(1, shocks)["_X"][1]
res = solved.compiled.equations(fwd, cur, {k: float(v) for k, v in params.items()})
print(res)

[ 0.02315669+0.j  0.55888162+0.j -0.25949515+0.j  0.        +0.j
  0.09450814+0.j  0.09825764+0.j]
